# Job Board Scraping Lab

In this lab you will first see a minimal but fully functional code snippet to scrape the LinkedIn Job Search webpage. You will then work on top of the example code and complete several chanllenges.

### Some Resources 

- [Requests library](http://docs.python-requests.org/en/master/#the-user-guide) documentation 
- [Beautiful Soup Doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Urllib](https://docs.python.org/3/library/urllib.html#module-urllib)
- [re lib](https://docs.python.org/3/library/re.html)
- [Scrapy](https://scrapy.org/)
- [List of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
- [HTML basics](http://www.simplehtmlguide.com/cheatsheet.php)
- [CSS basics](https://www.cssbasics.com/#page_start)

In [1]:
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search(keywords,location):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Assemble the full url with parameters
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords, "&location=", location])

    # Create a request to get the data from the server 
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for card in soup.select("div.result-card__contents"):
        title = card.findChild("h3", recursive=False)
        company = card.findChild("h4", recursive=False)
        location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
        titles.append(title.string)
        companies.append(company.string)
        locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

In [2]:
# Example to call the function

results = scrape_linkedin_job_search('data%20analyst', "Berlin")
results

,Title,Company,Location
0,Data Analyst (m/f/d),BASF,"Berlin, Berlin, Germany"
1,Junior Project Manager / Data analyst,Sony Electronics,"Berlin, Berlin, Germany"
2,(Junior) Data Scientist / Data Analyst (w/m/d),KPMG Deutschland,"Berlin, Berlin, Germany"
3,Junior Data Analyst (f/m/d),Delivery Hero,"Berlin, Berlin, Germany"
4,Junior Data Analyst (m/w/d),factorymarket,"Berlin, Berlin, Germany"
5,Game Data Analyst (f/m/d),Kolibri Games,"Berlin, Berlin, Germany"
6,Data Analyst,Jam City,"Berlin, Berlin, Germany"
7,Data/Business Analyst (m/f/d),Simplifa GmbH,"Berlin, Berlin, Germany"
8,Data Analyst,Harnham,"Berlin, Berlin, Germany"
9,Data Analyst (f/m/div),Penta Group,"Berlin, Berlin, Germany"


## Challenge 1

The first challenge for you is to update the `scrape_linkedin_job_search` function by adding a new parameter called `num_pages`. This will allow you to search more than 25 jobs with this function. Suggested steps:

1. Go to https://www.linkedin.com/jobs/search/?keywords=data%20analysis in your browser.
1. Scroll down the left panel and click the page 2 link. Look at how the URL changes and identify the page offset parameter.
1. Add `num_pages` as a new param to the `scrape_linkedin_job_search` function. Update the function code so that it uses a "for" loop to retrieve several pages of search results.
1. Test your new function by scraping 5 pages of the search results.

Hint: Prepare for the case where there are less than 5 pages of search results. Your function should be robust enough to **not** trigger errors. Simply skip making additional searches and return all results if the search already reaches the end.

In [3]:
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

# get the url
def get_url(keywords, location, start):
    BASE_URL = "https://www.linkedin.com/jobs/search/?"
    return ''.join([BASE_URL, 'keywords=', keywords, "&location=", location, "&start=", str(start)])


# get dataframe of one page
def get_one_page(url):
    # Create a request to get the data from the server 
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for card in soup.select("div.result-card__contents"):
        title = card.findChild("h3", recursive=False)
        company = card.findChild("h4", recursive=False)
        location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
        titles.append(title.string)
        companies.append(company.string)
        locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

In [4]:
get_url("data%20analyst", "Berlin", 2)

'https://www.linkedin.com/jobs/search/?keywords=data%20analyst&location=Berlin&start=2'

In [5]:
get_one_page('https://www.linkedin.com/jobs/search/?keywords=data%20analyst&location=Berlin&start=2')

,Title,Company,Location
0,Data Analyst (m/f/d),BASF,"Berlin, Berlin, Germany"
1,Game Data Analyst (f/m/d),Kolibri Games,"Berlin, Berlin, Germany"
2,Junior Data Analyst (m/w/d),factorymarket,"Berlin, Berlin, Germany"
3,(Junior) Data Scientist / Data Analyst (w/m/d),KPMG Deutschland,"Berlin, Berlin, Germany"
4,Junior Project Manager / Data analyst,Sony Electronics,"Berlin, Berlin, Germany"
5,Junior Data Analyst (m/w/d),The Retail Factory Pty Ltd,"Berlin, Berlin, Germany"
6,Data Analyst - Business Intelligence (m/f/div),Elinvar.de,"Berlin, Berlin, Germany"
7,Data Analyst (m/f/d),Cortado,"Berlin, Berlin, Germany"
8,Data Analyst Customer Experience (m|w|x),Axel Springer SE,"Berlin, Berlin, Germany"
9,Optimization Analyst,WeAreAspire,"Berlin, Berlin, Germany"


In [6]:
def get_jobs_linkedin(keywords, location, pages):
    #get list of url's using the function get_url and list comprehension
    list_of_urls = [get_url(keywords, location,i) for i in range(pages)]
    #get list of df from the list of urls
    list_of_data = [get_one_page(j) for j in list_of_urls]
    # concat list of data to return one single df
    data = pd.concat(list_of_data)
    return data

In [7]:
test = get_jobs_linkedin("data%20analyst", "Berlin", 6)


In [8]:
test

,Title,Company,Location
0,Data Analyst (m/f/d),BASF,"Berlin, Berlin, Germany"
1,Junior Project Manager / Data analyst,Sony Electronics,"Berlin, Berlin, Germany"
2,(Junior) Data Scientist / Data Analyst (w/m/d),KPMG Deutschland,"Berlin, Berlin, Germany"
3,Junior Data Analyst (f/m/d),Delivery Hero,"Berlin, Berlin, Germany"
4,Junior Data Analyst (m/w/d),factorymarket,"Berlin, Berlin, Germany"
...,...,...,...
20,Business Intelligence Engineer,Amazon,"Berlin, Berlin, Germany"
21,Data Analyst (m/w),IGES Institut GmbH,"Berlin, Berlin, Germany"
22,Data Analyst Customer Experience (m|w|x),Axel Springer SE,"Berlin, Berlin, Germany"
23,"Analyst, Marketing EU (m/f/d)",Casper,"Berlin, Berlin, Germany"


## Challenge 2

Further improve your function so that it can search jobs in a specific country. Add the 3rd param to your function called `country`. The steps are identical to those in Challange 1.

In [9]:
# Done above

## Challenge 3

Add the 4th param called `num_days` to your function to allow it to search jobs posted in the past X days. Note that in the LinkedIn job search the searched timespan is specified with the following param:

```
f_TPR=r259200
```

The number part in the param value is the number of seconds. 259,200 seconds equal to 3 days. You need to convert `num_days` to number of seconds and supply that info to LinkedIn job search.

In [10]:
# your code here

## Bonus Challenge

Allow your function to also retrieve the "Seniority Level" of each job searched. Note that the Seniority Level info is not in the initial search results. You need to make a separate search request for each job card based on the `currentJobId` value which you can extract from the job card HTML.

After you obtain the Seniority Level info, update the function and add it to a new column of the returned dataframe.

In [11]:
# your code here